In [2]:
RKP = "DL031"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set_theme(style="darkgrid")

In [3]:
# Load datasets and rename columns, load all aqi data but specify metro data name
def loadcsv(city="./data/rkpuram.csv"):
    met = pd.read_csv(city,delimiter=';',skiprows=24)
    aqi = pd.read_csv('./data/station_hour.csv')
    print(aqi.columns)
    met.rename(columns={'# Date': 'Date',}, inplace=True)
    met.rename(columns={'UT time': 'Time',}, inplace=True)
    aqi['Time'] = aqi['Datetime'].str[-8:-3]
    aqi['Date'] = aqi['Datetime'].str[0:10]
    stations = ["DL"+str(x).zfill(3) for x in range(1,39)]
    split_aqi = {}
    for i in range(len(stations)):
        split_aqi[stations[i]] = (aqi[aqi['StationId'] == stations[i]])
    return met,aqi,split_aqi
met,aqi,split_aqi = loadcsv()

C:\Users\confusement\miniconda3\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Index(['StationId', 'Datetime', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3',
       'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket'],
      dtype='object')


In [7]:
# Pre - processing and loading data
class dataset:
    def __init__(self,met,aqi,split_aqi):
            self.metro_data = met
            self.aqi_data = aqi
            self.split_aqi = split_aqi
    def mergedData(self,station,rlist=['PM2.5','PM10','NO','NO2','NOx','NH3','CO','SO2','O3'],roll=48,shift=48):
        df_aqi = self.getdf(station)
        df = pd.merge(df_aqi, self.metro_data, how='inner', on=['Date', 'Time'])
        print("Merged Dataset Size",len(df))
        
        #Pre Processing merged Data
        df['Year'] = df['Date'].str[0:4]
        df['Month'] = df['Date'].str[5:7].astype(np.float64)
        df['Day'] = df['Date'].str[8:10].astype(np.float64)
        df['Hour'] = df['Time'].str[0:2]
        
        # TRIG TRANSFORMATIONS
        df['windX'] = np.cos(np.deg2rad(df['Wind direction'])) * df['Wind speed']
        df['windY'] = np.sin(np.deg2rad(df['Wind direction'])) * df['Wind speed']
        df['hourX'] = np.cos((df['Hour'].astype(np.float64)-1)*np.pi/24)
        df['hourY'] = np.sin((df['Hour'].astype(np.float64)-1)*np.pi/24)
        df['MonthX'] = np.cos((df['Month'].astype(np.float64)-1)*np.pi/12)
        df['MonthY'] = np.sin((df['Month'].astype(np.float64)-1)*np.pi/12)
        
        import datetime
        df['Date'] = pd.to_datetime(df['Date'])
        df['isWeekend'] =  (df['Date'].dt.dayofweek>=5).astype(int)
        
        df.interpolate(method='linear', limit=5,inplace=True)
        
        # Drop Additional columns
        df.drop('Benzene', axis=1, inplace=True)
        df.drop('Toluene',axis=1, inplace=True)
        df.drop('Xylene', axis=1,inplace=True)
        df.drop('AQI_Bucket',axis=1,inplace=True)
        df.drop('Datetime',axis=1,inplace=True)
        df.drop('StationId',axis=1,inplace=True)
        df.drop('Short-wave irradiation',axis=1,inplace=True)
        df.drop('Date',axis=1,inplace=True)
        df.drop('Time',axis=1,inplace=True)
        
        # Rolling and shifting 
        print("Size before roll",len(df))
#         for i in rlist:
#             df[i+'_mean'] = df[i].rolling(window=roll, min_periods=int(roll//4)).mean()
#             df[i+'_min'] = df[i].rolling(window=roll, min_periods=int(roll/4)).min()
#             df[i+'_max'] = df[i].rolling(window=roll, min_periods=int(roll/4)).max()
        for i in rlist:
            df[i+'_min'] = df[i].shift(24)
            df[i+'_max'] = df[i].shift(48)
            df[i+'_mean'] = df[i].shift(72)
        for i in rlist:
            df[i] = df[i].shift(-shift)
        df.dropna(inplace=True)
        print("Size after roll",len(df))
        
        return df.copy()
    def getdf(self,station):
        return self.split_aqi[station]
    def plot(self,station):
        df = self.getdf(station)
    def stats(self):
        pass
dat = dataset(met,aqi,split_aqi)
df = dat.mergedData('DL031')

Merged Dataset Size 44035
Size before roll 44035
Size after roll 23372


In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression , mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
dfTrain = df[:]
X = df[['PM2.5_mean','PM2.5_min','PM2.5_max','PM10_mean','PM10_min','PM10_max','CO_mean','CO_min','CO_max','NO2_mean','NO2_min','NO2_max','Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']]
y = df['PM2.5']
scaler = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33, random_state=42)
print(scaler.fit(Xtrain))

# reg = LinearRegression().fit(scaler.transform(Xtrain), ytrain)
reg = MLPRegressor(random_state=1, max_iter=300).fit(scaler.transform(Xtrain), ytrain)
# reg = SVR(C=1.0, epsilon=0.2).fit(scaler.transform(Xtrain), ytrain)

testPred = reg.predict(scaler.transform(Xtest))
trainPred = reg.predict(scaler.transform(Xtrain))
mse = np.mean((testPred - np.array(ytest))*(testPred - np.array(ytest)))
print(reg.score(X, y))
print(mean_squared_error(testPred, ytest,squared=False))
print(mean_squared_error(trainPred, ytrain,squared=False))

StandardScaler()


In [8]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [9]:
from keras.models import Sequential
from keras.layers import Dense
with tf.device('/CPU:0'):
    model = Sequential()
    model.add(Dense(20, input_dim=22, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.summary()
    #Fit
    model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
    history = model.fit(scaler.transform(Xtrain), ytrain, epochs=1000, batch_size=50,  verbose=1, validation_split=0.2)
    #Print Accuracy
    testPred = model.predict(scaler.transform(Xtest))
    trainPred = model.predict(scaler.transform(Xtrain))
    print(mean_squared_error(testPred, ytest,squared=False))
    print(mean_squared_error(trainPred, ytrain,squared=False))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                460       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 901
Trainable params: 901
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
253/253 [==============================] - 1s 5ms/step - loss: 17451.5332 - mse: 17451.5332 - mae: 97.1036 - val_loss: 10284.6592 - val_mse: 10284.6592 - val_mae: 68.1212
Epoch 2/1000
253/253 [==============================] - 1s 3ms/step - loss: 7678.1660 - mse: 7678.1660 - mae: 59.0249 - val_loss: 7578.3496 - val_mse: 7578.3496 - val_mae: 56.9655
Epoch 3/1000
253/253 [==

253/253 [==============================] - 1s 2ms/step - loss: 4290.7725 - mse: 4290.7725 - mae: 42.7522 - val_loss: 5099.5474 - val_mse: 5099.5474 - val_mae: 45.2162
Epoch 88/1000
253/253 [==============================] - 1s 3ms/step - loss: 4299.3081 - mse: 4299.3081 - mae: 42.7895 - val_loss: 5096.2222 - val_mse: 5096.2222 - val_mae: 45.0249
Epoch 89/1000
253/253 [==============================] - 1s 2ms/step - loss: 4278.5483 - mse: 4278.5483 - mae: 42.7115 - val_loss: 5081.9712 - val_mse: 5081.9712 - val_mae: 45.0876
Epoch 90/1000
253/253 [==============================] - 1s 2ms/step - loss: 4275.1680 - mse: 4275.1680 - mae: 42.7208 - val_loss: 5076.1899 - val_mse: 5076.1899 - val_mae: 45.0802
Epoch 91/1000
253/253 [==============================] - 1s 2ms/step - loss: 4257.0884 - mse: 4257.0884 - mae: 42.5965 - val_loss: 5072.1299 - val_mse: 5072.1299 - val_mae: 45.1042
Epoch 92/1000
253/253 [==============================] - 1s 2ms/step - loss: 4256.9849 - mse: 4256.9849 - mae

253/253 [==============================] - 0s 2ms/step - loss: 3892.2224 - mse: 3892.2224 - mae: 40.8986 - val_loss: 4895.7915 - val_mse: 4895.7915 - val_mae: 44.5290
Epoch 178/1000
253/253 [==============================] - 0s 2ms/step - loss: 3896.1714 - mse: 3896.1714 - mae: 40.9272 - val_loss: 4907.2275 - val_mse: 4907.2275 - val_mae: 44.1566
Epoch 179/1000
253/253 [==============================] - 0s 2ms/step - loss: 3880.8259 - mse: 3880.8259 - mae: 40.7847 - val_loss: 4907.9619 - val_mse: 4907.9619 - val_mae: 44.5856
Epoch 180/1000
253/253 [==============================] - 0s 2ms/step - loss: 3878.2231 - mse: 3878.2231 - mae: 40.8454 - val_loss: 4900.5225 - val_mse: 4900.5225 - val_mae: 44.2672
Epoch 181/1000
253/253 [==============================] - 0s 2ms/step - loss: 3875.4055 - mse: 3875.4055 - mae: 40.8683 - val_loss: 4898.5376 - val_mse: 4898.5376 - val_mae: 44.3451
Epoch 182/1000
253/253 [==============================] - 0s 2ms/step - loss: 3871.6011 - mse: 3871.6011 

253/253 [==============================] - 0s 2ms/step - loss: 3619.7629 - mse: 3619.7629 - mae: 39.7590 - val_loss: 4799.0757 - val_mse: 4799.0757 - val_mae: 44.0146
Epoch 268/1000
253/253 [==============================] - 0s 2ms/step - loss: 3613.7483 - mse: 3613.7483 - mae: 39.7753 - val_loss: 4799.5894 - val_mse: 4799.5894 - val_mae: 43.9138
Epoch 269/1000
253/253 [==============================] - 1s 2ms/step - loss: 3628.8940 - mse: 3628.8940 - mae: 39.8836 - val_loss: 4820.5713 - val_mse: 4820.5713 - val_mae: 43.6718
Epoch 270/1000
253/253 [==============================] - 0s 2ms/step - loss: 3618.5471 - mse: 3618.5471 - mae: 39.7244 - val_loss: 4820.2036 - val_mse: 4820.2036 - val_mae: 43.8228
Epoch 271/1000
253/253 [==============================] - 0s 2ms/step - loss: 3606.2285 - mse: 3606.2285 - mae: 39.6793 - val_loss: 4803.3125 - val_mse: 4803.3125 - val_mae: 43.7588
Epoch 272/1000
253/253 [==============================] - 0s 2ms/step - loss: 3611.6824 - mse: 3611.6824 

253/253 [==============================] - 1s 2ms/step - loss: 3484.2546 - mse: 3484.2546 - mae: 39.1132 - val_loss: 4784.0625 - val_mse: 4784.0630 - val_mae: 43.8148
Epoch 358/1000
253/253 [==============================] - 1s 2ms/step - loss: 3487.1707 - mse: 3487.1707 - mae: 39.1843 - val_loss: 4795.2251 - val_mse: 4795.2251 - val_mae: 43.8269
Epoch 359/1000
253/253 [==============================] - 1s 2ms/step - loss: 3484.3127 - mse: 3484.3127 - mae: 39.0958 - val_loss: 4791.8389 - val_mse: 4791.8389 - val_mae: 44.1007
Epoch 360/1000
253/253 [==============================] - 1s 2ms/step - loss: 3484.4412 - mse: 3484.4412 - mae: 39.1557 - val_loss: 4811.4336 - val_mse: 4811.4336 - val_mae: 44.5660
Epoch 361/1000
253/253 [==============================] - 1s 2ms/step - loss: 3479.1433 - mse: 3479.1433 - mae: 39.1426 - val_loss: 4801.0913 - val_mse: 4801.0913 - val_mae: 43.7586
Epoch 362/1000
253/253 [==============================] - 1s 2ms/step - loss: 3482.4895 - mse: 3482.4895 

...

253/253 [==============================] - 0s 2ms/step - loss: 3276.5073 - mse: 3276.5073 - mae: 38.3511 - val_loss: 4746.7344 - val_mse: 4746.7344 - val_mae: 43.9961
Epoch 628/1000
253/253 [==============================] - 1s 2ms/step - loss: 3276.6609 - mse: 3276.6609 - mae: 38.3635 - val_loss: 4758.6655 - val_mse: 4758.6655 - val_mae: 43.8626
Epoch 629/1000
253/253 [==============================] - 0s 2ms/step - loss: 3291.5273 - mse: 3291.5273 - mae: 38.4091 - val_loss: 4749.2471 - val_mse: 4749.2471 - val_mae: 43.9892
Epoch 630/1000
253/253 [==============================] - 0s 2ms/step - loss: 3281.4817 - mse: 3281.4817 - mae: 38.3583 - val_loss: 4757.1914 - val_mse: 4757.1914 - val_mae: 43.8683
Epoch 631/1000
253/253 [==============================] - 0s 2ms/step - loss: 3274.0400 - mse: 3274.0400 - mae: 38.3609 - val_loss: 4746.2080 - val_mse: 4746.2080 - val_mae: 44.4062
Epoch 632/1000
253/253 [==============================] - 0s 2ms/step - loss: 3315.9939 - mse: 3315.9939 

253/253 [==============================] - 0s 2ms/step - loss: 3240.3862 - mse: 3240.3862 - mae: 38.2574 - val_loss: 4815.5327 - val_mse: 4815.5327 - val_mae: 44.0014
Epoch 718/1000
253/253 [==============================] - 0s 2ms/step - loss: 3254.4773 - mse: 3254.4773 - mae: 38.3207 - val_loss: 4746.2148 - val_mse: 4746.2148 - val_mae: 44.4374
Epoch 719/1000
253/253 [==============================] - 0s 2ms/step - loss: 3250.7915 - mse: 3250.7915 - mae: 38.2979 - val_loss: 4751.7627 - val_mse: 4751.7627 - val_mae: 43.8938
Epoch 720/1000
253/253 [==============================] - 0s 2ms/step - loss: 3243.0659 - mse: 3243.0659 - mae: 38.2659 - val_loss: 4746.9497 - val_mse: 4746.9497 - val_mae: 44.1799
Epoch 721/1000
253/253 [==============================] - 0s 2ms/step - loss: 3248.0635 - mse: 3248.0632 - mae: 38.3000 - val_loss: 4730.8936 - val_mse: 4730.8936 - val_mae: 44.0318
Epoch 722/1000
253/253 [==============================] - 0s 2ms/step - loss: 3247.7629 - mse: 3247.7629 

253/253 [==============================] - 1s 2ms/step - loss: 3218.5161 - mse: 3218.5161 - mae: 38.1821 - val_loss: 4714.6753 - val_mse: 4714.6753 - val_mae: 44.1337
Epoch 808/1000
253/253 [==============================] - 1s 2ms/step - loss: 3223.8696 - mse: 3223.8696 - mae: 38.1488 - val_loss: 4736.5996 - val_mse: 4736.5996 - val_mae: 44.5477
Epoch 809/1000
253/253 [==============================] - 0s 2ms/step - loss: 3209.0298 - mse: 3209.0298 - mae: 38.1236 - val_loss: 4726.0957 - val_mse: 4726.0957 - val_mae: 44.2284
Epoch 810/1000
253/253 [==============================] - 0s 2ms/step - loss: 3227.1243 - mse: 3227.1243 - mae: 38.2210 - val_loss: 4709.1538 - val_mse: 4709.1538 - val_mae: 44.0482
Epoch 811/1000
253/253 [==============================] - 0s 2ms/step - loss: 3214.7942 - mse: 3214.7942 - mae: 38.1578 - val_loss: 4739.7031 - val_mse: 4739.7031 - val_mae: 44.1894
Epoch 812/1000
253/253 [==============================] - 0s 2ms/step - loss: 3218.5042 - mse: 3218.5044 

253/253 [==============================] - 0s 2ms/step - loss: 3199.0242 - mse: 3199.0247 - mae: 38.1157 - val_loss: 4695.9966 - val_mse: 4695.9966 - val_mae: 44.0601
Epoch 898/1000
253/253 [==============================] - 0s 2ms/step - loss: 3188.8115 - mse: 3188.8115 - mae: 38.0777 - val_loss: 4726.6636 - val_mse: 4726.6636 - val_mae: 43.7640
Epoch 899/1000
253/253 [==============================] - 0s 2ms/step - loss: 3199.7063 - mse: 3199.7063 - mae: 38.0853 - val_loss: 4704.8906 - val_mse: 4704.8906 - val_mae: 44.0034
Epoch 900/1000
253/253 [==============================] - 0s 2ms/step - loss: 3187.1182 - mse: 3187.1182 - mae: 38.0710 - val_loss: 4703.5493 - val_mse: 4703.5493 - val_mae: 43.9248
Epoch 901/1000
253/253 [==============================] - 0s 2ms/step - loss: 3184.6265 - mse: 3184.6265 - mae: 38.0043 - val_loss: 4740.4897 - val_mse: 4740.4897 - val_mae: 44.3843
Epoch 902/1000
253/253 [==============================] - 0s 2ms/step - loss: 3194.9172 - mse: 3194.9172 

253/253 [==============================] - 0s 2ms/step - loss: 3151.4800 - mse: 3151.4800 - mae: 37.9933 - val_loss: 4751.6680 - val_mse: 4751.6680 - val_mae: 43.7220
Epoch 988/1000
253/253 [==============================] - 0s 2ms/step - loss: 3155.1760 - mse: 3155.1760 - mae: 37.9019 - val_loss: 4718.9409 - val_mse: 4718.9409 - val_mae: 43.7028
Epoch 989/1000
253/253 [==============================] - 0s 2ms/step - loss: 3147.3757 - mse: 3147.3757 - mae: 37.9693 - val_loss: 4728.7935 - val_mse: 4728.7935 - val_mae: 44.1589
Epoch 990/1000
253/253 [==============================] - 0s 2ms/step - loss: 3157.9880 - mse: 3157.9880 - mae: 37.9786 - val_loss: 4729.7510 - val_mse: 4729.7510 - val_mae: 44.3341
Epoch 991/1000
253/253 [==============================] - 0s 2ms/step - loss: 3155.2417 - mse: 3155.2417 - mae: 37.9782 - val_loss: 4724.0205 - val_mse: 4724.0205 - val_mae: 44.2863
Epoch 992/1000
253/253 [==============================] - 0s 2ms/step - loss: 3149.4658 - mse: 3149.4661 